In [13]:
import pulsar
import json
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output

In [14]:
client = pulsar.Client('pulsar://localhost:6650')

In [15]:
producer = client.create_producer('request')

2022-05-31 15:47:26.057 INFO  [140485261166400] ConnectionPool:85 | Created connection for pulsar://localhost:6650
2022-05-31 15:47:26.060 INFO  [140484219500288] ClientConnection:356 | [127.0.0.1:33858 -> 127.0.0.1:6650] Connected to broker
2022-05-31 15:47:26.075 INFO  [140484219500288] HandlerBase:54 | [persistent://public/default/request, ] Getting connection from pool
2022-05-31 15:47:26.084 INFO  [140484219500288] ProducerImpl:170 | [persistent://public/default/request, ] Created producer on broker [127.0.0.1:33858 -> 127.0.0.1:6650] 


In [16]:
total_data = 2000

In [17]:
test_stack = []

# Divides the request into batches and sends it to the analysis nodes
def makeRequest(data_size, batch_size=264):
    temp_total = data_size
    start_index = 0
    if not (0 < data_size < 365000):
        raise ValueError("Data Size not available!")
    while data_size > 0:
        if data_size - batch_size < 0:
            batch_size = data_size
        next_index = start_index
        next_index += batch_size
        producer.send((f"{start_index} - {next_index}").encode('utf-8'))
        #test_stack.append(f"{start_index} - {next_index}")
        start_index = next_index + 1
        data_size -= batch_size + 1

In [18]:
makeRequest(2000)

In [19]:
def aggregate_dicts(dict_a, dict_b):
    for key in dict_b.keys():
        if key not in dict_a.keys():
            dict_a[key] = int(dict_b[key])
        else:
            dict_a[key] += int(dict_b[key])
    return dict_a

def parse_dict(prog_dict):
    prog_df = pd.DataFrame.from_dict(prog_dict, orient='index', columns=["count"])
    prog_df = prog_df.sort_values("count", ascending=False)
    return prog_df

In [20]:
consumer_one = client.subscribe('response_one', subscription_name='client')
consumer_two = client.subscribe('response_two', subscription_name='client')
consumer_three = client.subscribe('response_three', subscription_name='client')
consumer_four = client.subscribe('response_four', subscription_name='client')

2022-05-31 15:47:29.621 INFO  [140485261166400] Client:88 | Subscribing on Topic :response_one
2022-05-31 15:47:29.627 INFO  [140484219500288] HandlerBase:54 | [persistent://public/default/response_one, client, 0] Getting connection from pool
2022-05-31 15:47:29.631 INFO  [140484219500288] ConsumerImpl:216 | [persistent://public/default/response_one, client, 0] Created consumer on broker [127.0.0.1:33858 -> 127.0.0.1:6650] 
2022-05-31 15:47:29.632 INFO  [140485261166400] Client:88 | Subscribing on Topic :response_two
2022-05-31 15:47:29.648 INFO  [140484219500288] HandlerBase:54 | [persistent://public/default/response_two, client, 1] Getting connection from pool
2022-05-31 15:47:29.653 INFO  [140484219500288] ConsumerImpl:216 | [persistent://public/default/response_two, client, 1] Created consumer on broker [127.0.0.1:33858 -> 127.0.0.1:6650] 
2022-05-31 15:47:29.653 INFO  [140485261166400] Client:88 | Subscribing on Topic :response_three
2022-05-31 15:47:29.661 INFO  [140484219500288]

In [ ]:
prog_dict = {}
top_repos = []
total_num = 0
total_test = 0
total_devop_test = 0

while True:
    # merge top ranking programming languages
    prog_lang = json.loads(consumer_one.receive().data().decode("utf-8"))
    prog_dict = aggregate_dicts(prog_dict, prog_lang)
    
    # List top commits
    top_repos.extend(json.loads(consumer_two.receive().data().decode("utf-8")))
    top_repos = sorted(top_repos, key=lambda tup: tup[1])
    top_repos = top_repos[:10]

    # Show total test
    numbers = consumer_three.receive().data().decode("utf-8").split(" ")
    total_test += int(numbers[0])
    total_num += int(numbers[2])
    
    # Show total test + devops
    numbers = consumer_three.receive().data().decode("utf-8").split(" ")
    total_devop_test += int(numbers[0])
    
    clear_output(wait=True)
    
    print(top_repos[:10])
    print(parse_dict(prog_dict).iloc[:10])
    print(f"{str(total_test)} / {str(total_num)}")
    print(f"{str(total_devop_test)} / {str(total_num)}")
    print(total_num)
    if total_num >= total_data:
        break

[['https://github.com/bestguy/svelte-solardial', 8], ['https://github.com/Hephaest/time-series-shapelet', 9], ['https://github.com/gregwhitaker/springboot-video-example', 9], ['https://github.com/Iduozmsn/goJava', 9], ['https://github.com/vegawidget/vegadown', 13], ['https://github.com/andiksetyawan/rest-api-boilerplate-go', 14], ['https://github.com/Ben2277/gosqltree', 16], ['https://github.com/bentasker/Golang-SSH-Tarpit', 17], ['https://github.com/EgorDm/NotionSy', 17], ['https://github.com/mamtajha-ts/k8s-4days', 17]]
                  count
Python              234
JavaScript          163
None                135
TypeScript          103
C++                  64
Java                 62
C#                   55
HTML                 54
Jupyter Notebook     52
Go                   41
206 / 1325
163 / 1325
1325
2022-05-31 16:27:26.078 INFO  [140484219500288] ProducerStatsImpl:92 | Producer [persistent://public/default/request, ] , ProducerStatsImpl (numMsgsSent_ = 0, numBytesSent_ = 0, sen

In [12]:
client.close()

2022-05-31 15:41:08.629 INFO  [140485261166400] ClientImpl:483 | Closing Pulsar client
2022-05-31 15:41:08.630 INFO  [140485261166400] ProducerImpl:546 | [persistent://public/default/request, standalone-0-43] Closing producer for topic persistent://public/default/request
2022-05-31 15:41:08.630 INFO  [140485261166400] ConsumerImpl:884 | [persistent://public/default/response_one, client, 0] Closing consumer for topic persistent://public/default/response_one
2022-05-31 15:41:08.630 INFO  [140485261166400] ConsumerImpl:884 | [persistent://public/default/response_two, client, 1] Closing consumer for topic persistent://public/default/response_two
2022-05-31 15:41:08.630 INFO  [140485261166400] ConsumerImpl:884 | [persistent://public/default/response_three, client, 2] Closing consumer for topic persistent://public/default/response_three
2022-05-31 15:41:08.631 INFO  [140485261166400] ConsumerImpl:884 | [persistent://public/default/response_four, client, 3] Closing consumer for topic persiste

In [25]:
a = [[1],[2]]
b = [[3]]
a.extend(b)
a

[[1], [2], [3]]